## Starting the 

In [13]:
import tabula
import pandas as pd

df_cerc = tabula.read_pdf("./input/LISTA-PARTICIPANTES-CERC-ARRANJOS-DE-PAGAMENTO-1.pdf", pages="all")

# Merge all dataframes of the list into one
df_cerc = pd.concat([df_cerc[0], df_cerc[1], df_cerc[2], df_cerc[3], df_cerc[4], df_cerc[5], df_cerc[6]])

# Transform the first row in header
df_cerc.columns = df_cerc.iloc[0]
df_cerc = df_cerc.drop(df_cerc.index[0])

# Drop the last column
df_cerc = df_cerc.drop(df_cerc.columns[-1], axis=1) 

# Remove the first column of the dataset
df_cerc = df_cerc.drop(df_cerc.columns[0], axis=1)

KeyError: 0

In [ ]:
df_cerc = tabula.read_pdf("./input/LISTA-PARTICIPANTES-CERC-ARRANJOS-DE-PAGAMENTO-1.pdf", pages="all")
len(df_cerc)

7

In [ ]:
df_cip_slc = tabula.read_pdf("./input/Participantes Homologados.pdf", pages="all")

df_cip_slc = df_cip_slc[0]

Got stderr: Aug 12, 2022 5:57:07 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 12, 2022 5:57:17 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [ ]:
df_cerc

,Razão Social,CNPJ
1,99 FOOD DELIVERY TECNOLOGIA LTDA,35.336.310/0001-66
2,99 Tecnologia,18.033.552/0001-61
3,A55 COMPANHIA SECURITIZADORA DE CREDITOS FINAN...,31.046.158/0001-26
4,A55 CONSULTORIA EM CRÉDITO LTDA,22.153.470/0001-28
5,Acqio Adquirencia S/A,33.171.211/0001-46
6,ADIQ,20.520.298/0001-78
7,Adyen do Brasil Ltda,14.796.606/0001-90
8,AGILLITAS,13.776.742/0001-55
9,AGL ADQUIRENCIA LTDA,29.150.228/0001-40
10,aiqfome.com,09.186.786/0001-58
